In [1]:
import pandas as pd
import sys
from goatools import obo_parser
from tqdm import tqdm
import os
import yaml
import h5py

In [65]:
sys.path.append("../")

In [66]:
from scripts.utils import convert_fasta_to_df

Убрать при предсказании сиквенсы имеющие разметку

In [3]:
train = pd.read_csv("../data/embeddings/input_csv/train_p2.csv")
pdb1000 = pd.read_csv("../data/embeddings/input_csv/pdb1000.csv")
pdb2272 = pd.read_csv("../data/embeddings/input_csv/pdb2272.csv")
pdb20000 = pd.read_csv("../data/embeddings/input_csv/pdb20000.csv")

In [4]:
train_ids = train["identifier"]
pdb1000_ids = pdb1000["identifier"]
pdb2272_ids = pdb2272["identifier"]
pdb20000_ids = pdb20000["identifier"]

In [25]:
path_go = "../data/go_terms/go-basic.obo"
go = obo_parser.GODag(path_go)

../data/go_terms/go-basic.obo: fmt(1.2) rel(2024-06-17) 45,494 Terms


In [26]:
na_term = "GO:0003676"

In [27]:
rec = go[na_term]
na_children_terms = rec.get_all_children()

In [28]:
na_children_terms = list(na_children_terms)

In [29]:
na_children_terms.append(na_term)

In [30]:
data = {'na_terms': na_children_terms}

In [34]:
# # Write the dictionary to a YAML file
# with open(file_path, 'w') as file:
#     yaml.dump(data, file, default_flow_style=False)

In [6]:
file_path = "../data/go_terms/na_terms.yml"

with open(file_path, 'r') as file:
    yaml_data = yaml.safe_load(file)

In [7]:
len(yaml_data['na_terms'])

312

In [84]:
def load_dict_from_hdf5(filename):
    """
    Load a dictionary with string keys and NumPy array values from an HDF5 file.

    Parameters:
    filename (str): Name of the HDF5 file to load the data from.

    Returns:
    dict: Dictionary with string keys and NumPy array values.
    """
    loaded_dict = {}
    with h5py.File(filename, "r") as f:
        for key in f.keys():
            loaded_dict[key] = f[key][:]
    return loaded_dict

In [ ]:
def save_dict_to_hdf5(data_dict, filename):
    """
    Save a dictionary with string keys and NumPy array values to an HDF5 file.

    Parameters:
    data_dict (dict): Dictionary with string keys and NumPy array values.
    filename (str): Name of the HDF5 file to save the data.
    """
    with h5py.File(filename, "w") as f:
        for key, value in data_dict.items():
            f.create_dataset(key, data=value)

In [85]:
neg_samples = load_dict_from_hdf5("../data/embeddings/ankh_embeddings/neg_samples_2d.h5")

In [87]:
embeddings = load_dict_from_hdf5("../data/embeddings/ankh_embeddings/train_p2_2d.h5")

In [ ]:
for key in neg_samples:
    embeddings[key] = neg_samples[key]